### stratified K 폴드 

stratified K 폴드는 불균형한 분포도를 가진 레이블 데이터 집합을 위한 K 폴드 방식. 
가령 특정 레이블 값이 특이하게 많거나 매우 적어서 값의 분포가 한쪽으로 치우치는 것.

In [21]:
import pandas as pd 
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold

iris = load_iris()
iris_df = pd.DataFrame(data=iris.data , columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

In [7]:
kfold = KFold(n_splits=3)
n_iter = 0

for train_index, test_index in kfold.split(iris_df):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('##교차 검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포 : \n', label_train.value_counts())
    print('검증 레이블 데이터 분포 : \n', label_test.value_counts())

##교차 검증 : 1
학습 레이블 데이터 분포 : 
 1    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 0    50
Name: label, dtype: int64
##교차 검증 : 2
학습 레이블 데이터 분포 : 
 0    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 1    50
Name: label, dtype: int64
##교차 검증 : 3
학습 레이블 데이터 분포 : 
 0    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    50
Name: label, dtype: int64


In [11]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter = 0

for train_index, test_index in skf.split(iris_df , iris_df['label']):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('##교차 검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포 : \n', label_train.value_counts())
    print('검증 레이블 데이터 분포 : \n', label_test.value_counts())
    

##교차 검증 : 1
학습 레이블 데이터 분포 : 
 2    34
0    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 0    17
1    17
2    16
Name: label, dtype: int64
##교차 검증 : 2
학습 레이블 데이터 분포 : 
 1    34
0    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 0    17
2    17
1    16
Name: label, dtype: int64
##교차 검증 : 3
학습 레이블 데이터 분포 : 
 0    34
1    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 1    17
2    17
0    16
Name: label, dtype: int64


## 교차 검증을 조금 더 편하게 해주는 API : cross_val_score()

cross_val_score() 은 KFold의 과정인 폴드세트 설정, for 루프에서 반복으로 학습 및 테스트 데이터의 인덱스 추출 및 반복적인 학습과 예측 수행 및 예측 성능 반환을 한 번에 수행.

corss_val_score() API로 내부에서 Estimator를 학습(fit), 예측(predict), 평가(evaluation) 시켜주므로 간단하게 교차 검증을 수행할 수 있슴. 

corss_val_score()는 하나의 평가 지표만 가능하지만, corss_validate() 는 여러 개의 평가 지표를 반환 가능 

보통 corss_val_score() 하나로도 대부분의 경우 쉽게 사용함.

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate 
from sklearn.datasets import load_iris
import numpy as np

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도 (accuracy) , 교차 검증 세트는 3개 
# cross_val_score 의 주요 파라미터 - estimator(Classifier or Regression), x (feature data set), y (label data set), 
# scoring (에측 성능 지표 ), cv(교차 검증 폴드 수)

scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)

print('교차 검증별 정확도: ', np.round(scores, 4))
print('평균 검증 정확도: ', np.round(np.mean(scores), 4))

교차 검증별 정확도:  [0.98 0.94 0.98]
평균 검증 정확도:  0.9667


# GridSearchCV - 교차 검증과 최적 하이퍼 파라미터 튜닝을 한 번에

sklearn 에서 GridSearchCV API를 이용해 Classifier 이나 Regressor과 같은 알고리즘에 사용되는 하이퍼 파라미터를 순차적으로 입력하면서 편리하게 최적의 파라미터를 도출할 수 있는 방안을 제공합니다. Grid는 격자라는 뜻으로 촘촘하게 파라미터를 순차적으로 변경하면서 최고 성능을 가지는 파라미터 조합을 찾고자 한다면 다음과 같이 파라미터의 집합을 만들고 이를 순차적으로 적용하면서 최적화 수행이 가능.

In [19]:
grid_parameters = {'max_depth: ': [1,2,3],
                  'min_depth' : [2,3]
                  }

### GridSearchCV 클래스의 생성자로 들어가는 주요 파라미터 

1. estimator : classifier, regressor, pipeline 
2. param_grid : key + 리스트 값을 가지는 딕셔너리가 주어짐. estimator의 튜닝을 위해 파라미터명과 사용될 여러 파라미터 값을 지정.
3. scoring : 예측 성능을 측정할 평가 방법을 지정합니다. 보통은 사이킷런의 성능 평가 지표를 지정하는 문자열(예- 정확도의 경우 'accuracy')로 지정하나 별도의 성능 평가 지표 함수도 지정할 수 있음.
4. cv : 교차 검증을 위해 분할되는 학습/테스트 세트의 개수를 지정 
5. refit : 디폴트가 True 이며, True로 생성 시 가장 최적의 하이퍼 파라미터를 찾은 뒤 입력된 estimator 객체를 해당 하이퍼 파라미터로 재학습 시킵니다. 

In [37]:
# 붓꽃 데이터를 이용해 GridSearchCV 실습 

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 데이터를 로딩하고 학습 데이터와 테스트 분리

iris = load_iris()
label = iris.target 
data = iris.data 

X_train, X_test, y_train, y_test = train_test_split(iris.data, label, test_size=0.2, random_state=121)

dtree = DecisionTreeClassifier()

### 파라미터를 딕셔너리 형태로 설정 

parameters = {'max_depth': [1,2,3], 'min_samples_split': [2,3]}

import pandas as pd

# param_grid 의 하이퍼 파라미터를 3개의 train, test set fold 롤 나누어 테스트 수행 설정.
### refit = True 가 default임. True 이면 가장 좋은 파라미터 설정으로 재학습시킴.

grid_dtree = GridSearchCV(dtree, param_grid = parameters, cv=3, refit=True)

# 붓꽃 학습 데이터로 param_grid 의 하이퍼 파리미터를 순차적으로 학습 / 평가

grid_dtree.fit(X_train, y_train)

# GridsearchCV 객체의 fit()을 수행하면 최고 성능을 나타낸 하이퍼 파라미터의 값과 그때의 평가 결과 값이 각각 best_params_ , best_score_ 속성에 기록됌.
# GridSearchCV 결과를 추출해 DataFrame 으로 변환 

scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params','mean_test_score','rank_test_score','split0_test_score','split1_test_score','split2_test_score']]

# cv_results_ : gridsearchcv 의 결과 세트로서 딕셔너리 형태로 key값과 리스트 형태의 value 값을 가집니다.

print('GridSearchCV 최적 파라미터: ', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도:{0:.4f} '.format(grid_dtree.best_score_))

# GridSearchCV의 refit으로 이미 학습된 estimator 반환 

estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 학습이 됐으므로 별도 학습이 필요 없음.

pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

GridSearchCV 최적 파라미터:  {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도:0.9750 
테스트 데이터 세트 정확도: 0.9667
